# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be archieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-09 10:17:58] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=30338, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=133759213, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_

[2025-04-09 10:18:08 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-09 10:18:08 TP0] Init torch distributed begin.


[2025-04-09 10:18:09 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-09 10:18:09 TP0] Load weight begin. avail mem=78.58 GB


[2025-04-09 10:18:09 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-09 10:18:10 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-04-09 10:18:13 TP0] Init torch distributed begin.
[2025-04-09 10:18:13 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-09 10:18:13 TP0] Load weight begin. avail mem=24.60 GB
[2025-04-09 10:18:14 TP0] Using model weights format ['*.bin']


Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]

[2025-04-09 10:18:15 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=23.67 GB, mem usage=0.93 GB.
[2025-04-09 10:18:15 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-04-09 10:18:15 TP0] Memory pool end. avail mem=23.35 GB
[2025-04-09 10:18:15 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096


[2025-04-09 10:18:16] INFO:     Started server process [1375434]
[2025-04-09 10:18:16] INFO:     Waiting for application startup.
[2025-04-09 10:18:16] INFO:     Application startup complete.
[2025-04-09 10:18:16] INFO:     Uvicorn running on http://127.0.0.1:30338 (Press CTRL+C to quit)
[2025-04-09 10:18:16] INFO:     127.0.0.1:49436 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-09 10:18:17] INFO:     127.0.0.1:34024 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-09 10:18:17 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-09 10:18:21 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-04-09 10:18:22] INFO:     127.0.0.1:34030 - "POST /generate HTTP/1.1" 200 OK
[2025-04-09 10:18:22] The server is fired up and ready to roll!


[2025-04-09 10:18:23] INFO:     127.0.0.1:34032 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-04-09 10:18:23] Child process unexpectedly failed with an exit code 9. pid=1375766
[2025-04-09 10:18:23] Child process unexpectedly failed with an exit code 9. pid=1375636


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-09 10:18:32] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=37557, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=424222707, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=False, log_r

[2025-04-09 10:18:44 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-09 10:18:44 TP0] Init torch distributed begin.


[2025-04-09 10:18:45 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-09 10:18:45 TP0] Load weight begin. avail mem=62.97 GB
[2025-04-09 10:18:46 TP0] Ignore import error when loading sglang.srt.models.llama4. 


[2025-04-09 10:18:47 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.64s/it]


...

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.30s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.30s/it]

[2025-04-09 10:18:52 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=35.13 GB, mem usage=1.04 GB.
[2025-04-09 10:18:52 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-04-09 10:18:52 TP0] Memory pool end. avail mem=34.82 GB


[2025-04-09 10:18:52 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-04-09 10:18:52] INFO:     Started server process [1378074]
[2025-04-09 10:18:52] INFO:     Waiting for application startup.
[2025-04-09 10:18:52] INFO:     Application startup complete.
[2025-04-09 10:18:52] INFO:     Uvicorn running on http://127.0.0.1:37557 (Press CTRL+C to quit)


[2025-04-09 10:18:53] INFO:     127.0.0.1:41022 - "GET /v1/models HTTP/1.1" 200 OK
[2025-04-09 10:18:53] INFO:     127.0.0.1:41034 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-09 10:18:53 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 10:18:57] INFO:     127.0.0.1:41036 - "POST /generate HTTP/1.1" 200 OK
[2025-04-09 10:18:57] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-09 10:18:58 TP0] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 10:18:58] INFO:     127.0.0.1:48556 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-04-09 10:18:58] Child process unexpectedly failed with an exit code 9. pid=1378681
[2025-04-09 10:18:58] Child process unexpectedly failed with an exit code 9. pid=1378614


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-09 10:19:06] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=33806, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=45506832, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_r

[2025-04-09 10:19:07] Casting torch.bfloat16 to torch.float16.


[2025-04-09 10:19:15 TP0] Casting torch.bfloat16 to torch.float16.


[2025-04-09 10:19:16 TP0] Casting torch.bfloat16 to torch.float16.
[2025-04-09 10:19:16 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-09 10:19:16 TP0] Init torch distributed begin.


[2025-04-09 10:19:16 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-09 10:19:16 TP0] Load weight begin. avail mem=61.30 GB
[2025-04-09 10:19:16 TP0] Ignore import error when loading sglang.srt.models.llama4. 


...

[2025-04-09 10:19:31 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192


[2025-04-09 10:19:32] INFO:     Started server process [1380695]
[2025-04-09 10:19:32] INFO:     Waiting for application startup.
[2025-04-09 10:19:32] INFO:     Application startup complete.
[2025-04-09 10:19:32] INFO:     Uvicorn running on http://127.0.0.1:33806 (Press CTRL+C to quit)


[2025-04-09 10:19:32] INFO:     127.0.0.1:35718 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-09 10:19:33] INFO:     127.0.0.1:35734 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-09 10:19:33 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 10:19:37] INFO:     127.0.0.1:35742 - "POST /generate HTTP/1.1" 200 OK
[2025-04-09 10:19:37] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-09 10:19:37 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-09 10:19:38] INFO:     127.0.0.1:50964 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-04-09 10:19:38] Child process unexpectedly failed with an exit code 9. pid=1381192
[2025-04-09 10:19:38] Child process unexpectedly failed with an exit code 9. pid=1381122


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-09 10:19:47] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=33619, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=677742805, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests=

[2025-04-09 10:19:55 TP0] Casting torch.bfloat16 to torch.float16.


[2025-04-09 10:19:56 TP0] Casting torch.bfloat16 to torch.float16.
[2025-04-09 10:19:56 TP0] Attention backend not set. Use flashinfer backend by default.
[2025-04-09 10:19:56 TP0] Init torch distributed begin.


[2025-04-09 10:19:58 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-09 10:19:58 TP0] Load weight begin. avail mem=61.30 GB


[2025-04-09 10:19:59 TP0] Ignore import error when loading sglang.srt.models.llama4. 


...

[2025-04-09 10:20:14 TP0] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.02it/s]

[2025-04-09 10:20:14 TP0] Load weight end. type=LlamaForCausalLMEagle3, dtype=torch.float16, avail mem=24.70 GB, mem usage=1.77 GB.
[2025-04-09 10:20:14 TP0] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-04-09 10:20:14 TP0] Memory pool end. avail mem=24.62 GB


[2025-04-09 10:20:15 TP0] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-09 10:20:15] INFO:     Started server process [1382947]
[2025-04-09 10:20:15] INFO:     Waiting for application startup.
[2025-04-09 10:20:15] INFO:     Application startup complete.
[2025-04-09 10:20:15] INFO:     Uvicorn running on http://127.0.0.1:33619 (Press CTRL+C to quit)
[2025-04-09 10:20:15] INFO:     127.0.0.1:53328 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-09 10:20:16] INFO:     127.0.0.1:47594 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-09 10:20:16 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-04-09 10:20:20 TP0] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-09 10:20:20] INFO:     127.0.0.1:47602 - "POST /generate HTTP/1.1" 200 OK
[2025-04-09 10:20:20] The server is fired up and ready to roll!


[2025-04-09 10:20:21] INFO:     127.0.0.1:47616 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-04-09 10:20:21] Child process unexpectedly failed with an exit code 9. pid=1383173
[2025-04-09 10:20:21] Child process unexpectedly failed with an exit code 9. pid=1383107


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionaly to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).